



#**Aivle 스쿨 지원 질문, 답변 챗봇 만들기**
# 단계3 : 모델링

## 0. 미션

* 워드 임베딩 벡터 기반 머신러닝 분류 모델링
    * 워드 임베딩 모델을 만들고 임베딩 벡터를 생성합니다.
    * 임베딩 벡터를 이용하여 intent를 분류하는 모델링을 수행합니다.
    * 예측된 intent의 답변 중 임의의 하나를 선정하여 출력합니다.
* 챗봇 : 모델의 예측결과(intent)에 따라 답변하는 챗봇 만들기
    * 질문을 입력받아, 답변하는 함수를 생성합니다.

## 1. 환경준비

### (1) 라이브러리 설치

#### 1) 자연어 처리를 위한 라이브러리

In [1]:
#gensim은 자연어 처리를 위한 오픈소스 라이브러리입니다. 토픽 모델링, 워드 임베딩 등 다양한 자연어 처리 기능을 제공
!pip install gensim

#### 2) 형태소 분석을 위한 라이브러리
* 참조 : https://konlpy.org/en/latest/install/

In [2]:
# mecab 설치를 위한 관련 패키지 설치
!apt-get install curl git
!apt-get install build-essential
!apt-get install cmake
!apt-get install g++
!apt-get install flex
!apt-get install bison
!apt-get install python-dev
!pip install cython
!pip install mecab-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.16).
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
B

In [3]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu: linux, mac os 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# !pip install mecab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 36.9 MB/s eta 0:00:00
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   506k      0  0:00:02  0:00:02 --:--:--  918k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/example.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.c
mecab-0.996-ko-0.9.2/example/example.c
mecab-0.996-ko-0.9.2/example/thread_test.cpp
mecab-0.996-ko-0.9.2/mecab-config.in
mecab-0.996-ko-0.9.2/man/
mecab-0.996-ko-0.9.2/man/Makefile.am
mecab-0.996-ko-0.9.2/man/mecab.1
mecab-0.996-ko-0.9.2/man/Makefile.in
mecab-0.996-ko-0.9.2/mecab.iss.in
mecab-0.996-ko-0.9.2/confi

### (2) 데이터 로딩
* 전처리 단계에서 생성한 데이터들을 로딩합니다.
    * train, val, test
* Google Colab 환경에서 진행을 권장합니다.
    * 구글 드라이브 바로 밑에 project 폴더를 만들고,
    * 데이터 파일을 복사해 넣습니다.

#### 1) Google Colab 환경 구축

* 구글 드라이브 연결

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd /content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차


#### 2) 저장된 데이터 읽어오기
* 저장된 파일들을 불러옵니다.

In [6]:
import joblib

train = joblib.load('train')
val = joblib.load('val')
test = joblib.load('test')

## 2. 전처리

### (1) 라벨 조정 및 배열화

In [7]:
train['intent'] = train['intent'] - 1
val['intent'] = val['intent'] - 1
test['intent'] = test['intent'] - 1

X_train = train['check'].values
y_train = train['intent'].values
X_val = val['check'].values
y_val = val['intent'].values
X_test = test['check'].values
y_test = test['intent'].values

### (2) 텍스트 정제 및 Okt 토큰화
* 텍스트 정규화
* 어간 추출
* 불용어 제거

In [8]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

# 다양한 토크나이저를 사용할 수 있는 함수
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        tokenizer = Kkma()

    return tokenizer

In [17]:
import re

def tokenize(tokenizer_name, original_sent, nouns=False):
    tokenizer = get_tokenizer(tokenizer_name)

    cleaned_sent = re.sub(r'[^\sa-zA-Z가-힣]', '', original_sent).lower().strip()
    cleaned_sent = re.sub(r'\s+', ' ', cleaned_sent)

    tokens = tokenizer.morphs(cleaned_sent, norm=True, stem=True)
    stop_words = ['mmm', 'q']
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

In [ ]:
tokenizer_name = "okt"
X_train_tokens = [tokenize(tokenizer_name, text) for text in X_train]
X_val_tokens = [tokenize(tokenizer_name, text) for text in X_val]
X_test_tokens = [tokenize(tokenizer_name, text) for text in X_test]

### (3) 수치화
* Keras Tokenizer
* 텍스트 데이터를 숫자 시퀀스로 변환

In [11]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train_tokens)

X_train_sequences = tokenizer.texts_to_sequences(X_train_tokens)
X_val_sequences = tokenizer.texts_to_sequences(X_val_tokens)
X_test_sequences = tokenizer.texts_to_sequences(X_test_tokens)

### (4) 패딩
* Keras Tokenizer
* 최대 시퀀스 길이 계산
* 텍스트 데이터를 숫자 시퀀스로 변환

In [12]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

all_tokens = X_train_tokens + X_val_tokens + X_test_tokens

max_sequence_length = max(len(sequence) for sequence in all_tokens)

X_train_pad = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_val_pad = pad_sequences(X_val_sequences, maxlen=max_sequence_length)
X_test_pad = pad_sequences(X_test_sequences, maxlen=max_sequence_length)

## 3. 모델링

### (1) 모델 1
* lstm

In [13]:
import numpy as np

embedding_dim = 300
vocab_size = len(tokenizer.word_index) + 1
num_classes = len(np.unique(y_train))

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# LSTM Model
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_sequence_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

batch_size = 8
epochs = 50

checkpoint = ModelCheckpoint("best_model.h5", monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')

early_stop = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

history = model.fit(X_train_pad, y_train, batch_size=batch_size, epochs=epochs,
                    validation_data=(X_val_pad, y_val), callbacks=[checkpoint, early_stop])

Epoch 1/50
130/130 [==============================] - ETA: 0s - loss: 3.7001 - accuracy: 0.1050
Epoch 1: val_accuracy improved from -inf to 0.15385, saving model to best_model.h5
130/130 [==============================] - 21s 138ms/step - loss: 3.7001 - accuracy: 0.1050 - val_loss: 3.2921 - val_accuracy: 0.1538
Epoch 2/50
130/130 [==============================] - ETA: 0s - loss: 2.7271 - accuracy: 0.2977
Epoch 2: val_accuracy improved from 0.15385 to 0.48462, saving model to best_model.h5
130/130 [==============================] - 18s 136ms/step - loss: 2.7271 - accuracy: 0.2977 - val_loss: 2.1557 - val_accuracy: 0.4846
Epoch 3/50
130/130 [==============================] - ETA: 0s - loss: 1.7358 - accuracy: 0.5549
Epoch 3: val_accuracy improved from 0.48462 to 0.62308, saving model to best_model.h5
130/130 [==============================] - 18s 136ms/step - loss: 1.7358 - accuracy: 0.5549 - val_loss: 1.5893 - val_accuracy: 0.6231
Epoch 4/50
130/130 [==============================] - E

In [ ]:
from tensorflow.keras.models import load_model

best_model = load_model("best_model.h5")

accuracy = best_model.evaluate(X_test_pad, y_test)[1]
print("Best Model Test Accuracy:", accuracy)

5/5 [==============================] - 0s 25ms/step - loss: 0.6613 - accuracy: 0.8462
Best Model Test Accuracy: 0.8461538553237915


In [ ]:
best_model.save("model.h5")

## 4. 챗봇
* **상세요구사항**
    * 챗봇 flow : input 질문 -> 분류 모델로 intent 예측 --> intent에 해당하는 답변 출력
        * 하나의 intent 에는 여러 답변이 있습니다. 이중 한가지를 랜덤하게 선택합니다.

#### 1) 사용자 입력 테스트

In [14]:
import numpy as np
from grammer_checker import spell_check
from tensorflow.keras.models import load_model

model = load_model("model.h5")


# 입력 전처리 및 예측 함수
def predict_answer(model, input_text):
    checked_text = spell_check(input_text)
    tokens = tokenize("okt", checked_text)
    sequences = tokenizer.texts_to_sequences([tokens])
    padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)
    prediction = model.predict(padded_sequences)
    predicted_class = np.argmax(prediction)
    answer = train.loc[train['intent']==predicted_class]['A'].unique()[0]
    return answer

In [18]:
predict_answer(model,"지원 조건 궁금해요")

1/1 [==============================] - 0s 27ms/step


'KT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자 중 만 34세 이하 미취업자를 대상으로 하는 교육입니다.\n단, 모집시점에 만 35세여도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다.\n또한 전공의 종류와는 무관 합니다.'

#### 2) 챗봇 함수 만들기
* 테스트 코드를 바탕으로 질문을 받아 답변을 하는 함수를 생성합시다.
* 성능이 좋은 모델 사용.

In [16]:
# 대화 함수
def chatbot():
    print("챗봇과 대화를 시작합니다. 종료하려면 '종료'라고 입력하세요.\n")
    while True:
        user_input = input("사용자: ")
        if user_input == '종료':
            print("챗봇과의 대화를 종료합니다.")
            break
        answer = predict_answer(model, user_input)
        print(f"챗봇: {answer}", '\n\n==============================================\n')

In [21]:
chatbot()

챗봇과 대화를 시작합니다. 종료하려면 '종료'라고 입력하세요.

사용자: 학교 수업과 교육을 병행할 수 있어?
1/1 [==============================] - 0s 26ms/step
챗봇: KT 에이블스쿨은 풀타임(09:00~18:00)으로 교육이 진행되며, 정해진 시간에 필수로 참여해야 합니다. 
교육에 풀타임으로 참여할 수 있어야 교육 수강이 가능합니다. 


사용자: 온라인 교육인가요?
1/1 [==============================] - 0s 37ms/step
챗봇: KT 에이블스쿨은 전국 어디서나 참여할 수 있도록, 온/오프라인을 병행하여 탄력적으로 운영합니다.
오프라인 교육장소는 6개 권역(수도권, 강원권, 부산/경남권, 대구/경북권, 충남/충북권, 전남/전북권)에 위치하고 있습니다. 

서류 접수 마감 이후에 지원 지역(교육 장소) 변경은 불가능하며, 최종 합격된 지역에서만 교육이 가능합니다. 


사용자: 온라인 교육인가요?
1/1 [==============================] - 0s 28ms/step
챗봇: KT 에이블스쿨은 전국 어디서나 참여할 수 있도록, 온/오프라인을 병행하여 탄력적으로 운영합니다.
오프라인 교육장소는 6개 권역(수도권, 강원권, 부산/경남권, 대구/경북권, 충남/충북권, 전남/전북권)에 위치하고 있습니다. 

서류 접수 마감 이후에 지원 지역(교육 장소) 변경은 불가능하며, 최종 합격된 지역에서만 교육이 가능합니다. 


사용자: 나 오늘 상 탔어요!
1/1 [==============================] - 0s 41ms/step
챗봇: 축하드려요! 


사용자: 핸드폰이 고장났어
1/1 [==============================] - 0s 23ms/step
챗봇: AS센터에 맡겨보세요. 


사용자: 산책 갈까?
1/1 [==============================] - 0s 27ms/step
챗봇: 좋은 생각입니다. 


사용자:

* **
* 최종 정확도 : 84%